In [2]:
#Create text file with links (EXECUTE THE NEXT 2 CELLS ONLY ONCE)
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

anime = []

for page in tqdm(range(0, 400)):
    url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup.find_all('tr'):
        links = tag.find_all('a')
        for link in links:        
            if type(link.get('id')) == str and len(link.contents[0]) > 1:
                anime.append((link.contents[0], link.get('href')) )


100%|██████████| 400/400 [05:13<00:00,  1.27it/s]


In [3]:
f = open("links.txt", "w")
for el in anime:
    f.write(str(el[1])+'\n')
f.close()
#We get 19083 rows, which implies we have less than 20000 animes

In [1]:
#Download the pages
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from random import randint
#multiprocessing library

#first open file from which we get links
f = open("links.txt","r")
lines = f.readlines()

for page in tqdm(range(0,100)):
    #Manage the directories
    os.system("mkdir Page"+str(page+1)) #the (page+1) is to make so we don't have the first element 0 but 1
    os.chdir("Page"+str(page+1))
    
    #These if conditions are needed to have the right indexing of the pages downloaded
    if page%2==0: start=page*50
    if page%2!=0: start=page*50+50
    
    for i in range(start,start+50):
        url=lines[i]
        check = requests.get(url)
        
        #Check if the site is blocking the download, start again when we are allowed to
        if check.status_code != 200:
            print("Last well executed page is {}".format(page-1))
            while(check.status_code != 200): #Stop while we can't download
                check = requests.get(url)
                #next line was just a try of changing the ip when the site would stop the downloads
                #in short didn't manage to implement it
                #os.system(("sudo ifconfig lo 127.0.0.{}  netmask 255.0.0.0").format(randint(0,50))) 
       
        #Save the content of the page 
        art=open("article"+str(i+1)+'.html',"wb")
        art.write(check.content)
        art.close()
    
    os.chdir("..")

 16%|█▌        | 16/100 [12:21<1:11:54, 51.37s/it]

Last well executed page is 15
Last well executed page is 15
Last well executed page is 15
Last well executed page is 15
Last well executed page is 15


 54%|█████▍    | 54/100 [41:28<34:18, 44.76s/it]  

Last well executed page is 53
Last well executed page is 53
Last well executed page is 53
Last well executed page is 53


 91%|█████████ | 91/100 [1:12:05<06:37, 44.15s/it]

Last well executed page is 90
Last well executed page is 90
Last well executed page is 90


100%|██████████| 100/100 [1:21:24<00:00, 48.85s/it]


In [1]:
import re
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from random import randint
from datetime import datetime

def Parser(art, tsv, soup):
    
    finder1,finder2,finder3,finder4,finder5,finder6,finder7,finder8,finder9,finder10,finder11,finder12,finder13,finder14,finder15 = "","","","","","","","","","","","","","",""
    
    finder1=str(soup.h1.contents).replace("</strong>","").replace("<strong>","")
    animeTitle=finder1 
    #print(finder1)
    #OK
    
    finder2=soup.select_one("a[href*=type]").contents[0].strip()
    animeType=finder2
    #print(finder2)
    #OK
        
    #finder3=int(soup.find_all('div', {'class':"spaceit_pad"})[4].getText().split()[1])
    for i in range(len(soup.find("div",{"style":"width: 225px"}).find_all("span",{"class":"dark_text"}))):
        if (soup.find("div",{"style":"width: 225px"}).find_all("div",{"class":"spaceit_pad"})[i].contents[1].getText())=="Episodes":
            finder3=int(soup.find("div",{"style":"width: 225px"}).find_all("div",{"class":"spaceit_pad"})[3].getText().strip().split()[1])
    animeNumEpisode = finder3
    #print(finder3)
    #OK
    
    start=""
    end=""
    finder4=soup.find_all('div', {'class':"spaceit_pad"})[6].getText().split()[1:]
    if len(finder4)>3:
        start=datetime.strptime(" ".join(finder4[:3]),"%b %d, %Y")
        end=datetime.strptime(" ".join(finder4[4:]), "%b %d, %Y")
    else:
        start=datetime.strptime(" ".join(finder4[:3]),"%b %d, %Y")
    
    releaseDate=start
    endDate=end        
    #OK
    
    #finder5=soup.find_all('div', {'class':"spaceit_pad"})[5].getText().strip().split("\n")[1].replace(" ","")
    #endDate = finder5
    #print(finder5)
    
    
    finder6=int(soup.find("span",{"class":"numbers members"}).getText().replace("Members ", "").replace(",",""))
    animeNumMembers = finder6
    #print(finder6)
    #OK
    
    finder7=float(soup.find("div",{"class":"score-label score-9"}).getText())
    animeScore = finder7
    #print(finder7)
    #OK
    
    
    finder8=int(soup.find("span",{"itemprop":"ratingCount","style":"display: none"}).getText())
    animeUsers = finder8
    #print(finder8)
    #OK
    
    finder9=int(soup.find("span", {"class":"numbers ranked"}).find("strong").getText().replace("#",""))
    animeRank = finder9
    #print(finder9)
    #OK
    
    finder10=int(str(soup.find(class_="numbers popularity").text).split('#')[-1])  
    animePopularity = finder10
    #print(finder10)
    #OK
    
    finder11=soup.find("p",{"itemprop":"description"}).getText()
    animeDescription = finder11.replace("/n","")
    #print(finder11)
    #OK
    
    finder12=[]
    for i in range(len(soup.find_all("li",{"class":"btn-anime","style":"width:90px"}))):
        finder12.append((soup.find_all("li",{"class":"btn-anime","style":"width:90px"}))[i]["title"])
    animeRelated = finder12
    #print(finder12)
    #OK
    
    finder13=[]
    finder14=[]
    for i in range(len(soup.find("div",{"class":"detail-characters-list clearfix"}).find_all("img"))):
        if i%2==0:
            finder13.append(",".join(soup.find("div",{"class":"detail-characters-list clearfix"}).find_all("img")[i]["alt"].replace(" ","").split(",")))
        else:
            finder14.append(",".join(soup.find("div",{"class":"detail-characters-list clearfix"}).find_all("img")[i]["alt"].replace(" ","").split(",")))
    
    animeCharacters = finder13
    animeVoices = finder14
    #OK
    #print(finder13)
    #print(finder14)
    
    finder15=[]
    for i in range(len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img"))):
        finder15.append(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")[i]["alt"])
        finder15.append((soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("div", {"class":"spaceit_pad"}))[i].getText().strip())
    l=[]
    for i in range(0,len(soup.find_all("div",{"class":"detail-characters-list clearfix"})[1].find_all("img")),1):
        l.append(finder15[i:i+2])
    animeStaff = l
    #print(finder15)
    
    tsv.write(animeTitle+"\t"+animeType+"\t"+str(animeNumEpisode)+"\t"+str(releaseDate)+"\t"+str(endDate)+"\t"+str(animeNumMembers)+"\t"+str(animeScore)+"\t"+str(animeUsers)+"\t"+str(animeRank)+"\t"+str(animePopularity)+"\t"+animeDescription+"\t"+str(animeRelated)+"\t"+str(animeCharacters)+"\t"+str(animeVoices)+"\t"+str(animeStaff))
    

In [25]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

#os.chdir("..")

#os.chdir("Page1")
art=open("article3.html")
tsv=open("anime_3.tsv","w")
soup=BeautifulSoup(art, "html.parser")                                   
#Parser(art,tsv,soup)
#<div style="width: 225px">
#for i in range(len(len(soup.find_all('div', {'class':"spaceit_pad"})))
#    if len(soup.find_all('div', {'class':"spaceit_pad"})[i].getText().split()[1:])==7:
#        print("Ok")

soup.find_all('div', {'class':"spaceit_pad"})[5].contents
#os.chdir("..")

['\n',
 <span class="dark_text">Aired:</span>,
 '\n  Apr 29, 2019 to Jul 1, 2019\n  ']

In [2]:
#PARSING
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm


for page in tqdm(range(0,10)):
    #Manage the directories
    
    
    #These if conditions are needed to have the right indexing of the pages downloaded
    if page%2==0: start=page*50
    if page%2!=0: start=page*50+50
    
    for i in range(start,start+50):
        #Open the articles of the page 
        os.chdir("Page"+str(page+1))
        art=open("article"+str(i+1)+'.html',"r")
        os.chdir("..")
        os.chdir("TSV")
        tsv=open("anime_"+str(i+1)+".tsv","w")
        
        soup=BeautifulSoup(art.read(), "html.parser") 
        
        Parser(art,tsv,soup)
        
        art.close()
        tsv.close()
        os.chdir("..")
    

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: time data 'Spring 2019' does not match format '%b %d, %Y'